In [105]:
import requests
import pandas as pd
from io import BytesIO
import os
from io import StringIO

In [106]:
# https://www.dataestur.es/en/apidata/

In [108]:
# Añadimos las urls de las que necesitamos descargar archivos. Key = Name; Value = URL

downloads = {
    "FRONTUR_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/FRONTUR_DL?desde%20%28a%C3%B1o%29=2018&Pa%C3%ADs%20de%20residencia=Todos&Tipo%20de%20visitante=Todos&CCAA%20de%20destino=Todos",
    "TURISMO_INTERNO_PROV_CCAA_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/TURISMO_INTERNO_PROV_CCAA_DL?CCAA%20de%20residencia=Todos&CCAA%20de%20destino=Todos",
    "GASTO_TPV_DESTINO_CCAA_MES_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/GASTO_TPV_DESTINO_CCAA_MES_DL?desde%20(a%C3%B1o)=2018",}
   # "CST_GASTO_CONSUMO_TURISTICO_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/CST_GASTO_CONSUMO_TURISTICO_DL?Producto=Todos"
   # "EGATUR_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/EGATUR_DL?desde%20%28a%C3%B1o%29=2018&Pa%C3%ADs%20de%20residencia=Todos&CCAA%20de%20destino=Todos",
   # "ACTIVIDADES_OCIO_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/ACTIVIDADES_OCIO_DL?CCAA=Todos&Provincia=Todos",
   # "TRANSPORTE_TERRESTRE_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/TRANSPORTE_TERRESTRE_DL?desde%20%28a%C3%B1o%29=2018&Tipo%20transporte=Todos",
   # "CONECTIVIDAD_AEREA_BUSQUEDAS_INICIO_VIAJE_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/CONECTIVIDAD_AEREA_BUSQUEDAS_INICIO_VIAJE_DL?Pa%C3%ADs%20origen=Todos&Ciudad%20destino=Todos&Tipo%20origen=Todos",
   # "ESCUCHA_ACTIVA_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/ESCUCHA_ACTIVA_DL?Idioma=Todos&Medio=Todos&Sentimiento=Todos",

In [109]:
# Creamos una función para descargar un diccionario de URLS. Para ello tenemos que pasarle la url y un nombre de archivo.
# La descarga devuelve un tipo binario, así que usamos BytesIO y lo convertimos en Dataframe con Pandas.

def download_excel(url, file_name, folder="./data"):
    file_path = os.path.join(folder, f"{file_name}.xlsx")

    if os.path.exists(file_path):
        print(f"- {file_name}.xlsx already exists.")
        return pd.read_excel(file_path)

    response = requests.get(url)
    
    if response.status_code == 200:
        file_type = response.headers["Content-Disposition"].split(".")[-1]

        if file_type == "xlsx":
            df = pd.read_excel(BytesIO(response.content))
            df.to_excel(file_path)
            print(f"{file_name}.xlsx downloaded and saved.")
            return df
        elif file_type == "csv":
            encoding = response.apparent_encoding
            df = pd.read_csv(StringIO(response.content.decode(encoding)), sep=';', engine='python')
            df.to_excel(file_path)
            print(f"{file_name}.csv downloaded and saved.")
            return df
    else:
        print(f"Failed to download {file_name}: {response.status_code}")
        return None

In [110]:
# Guardamos todos los dataframes. Llamamos nuestra función para crear un nuevo diccionario de dataframes.

datasets = {}

for name, url in downloads.items():
    df = download_excel(url, name)
    if df is not None:
        datasets[name] = df

- FRONTUR_DL.xlsx already exists.
- TURISMO_INTERNO_PROV_CCAA_DL.xlsx already exists.
- GASTO_TPV_DESTINO_CCAA_MES_DL.xlsx already exists.


In [111]:
#Para visualizar datos en conjunto:

for name, df in datasets.items():
    print(f"\n {name}")
    print("Columns:", df.columns.tolist())
    print("First 3 rows:\n", df.head(1))


 FRONTUR_DL
Columns: ['Unnamed: 0', 'AÑO', 'MES', 'PAIS_RESIDENCIA', 'TIPO_VISITANTE', 'CCAA_DESTINO', 'VISITANTES']
First 3 rows:
    Unnamed: 0   AÑO  MES PAIS_RESIDENCIA TIPO_VISITANTE CCAA_DESTINO  \
0           0  2018    1        Alemania  Excursionista    Andalucía   

   VISITANTES  
0         818  

 TURISMO_INTERNO_PROV_CCAA_DL
Columns: ['Unnamed: 0', 'AÑO', 'MES', 'CCAA_ORIGEN', 'PROVINCIA_ORIGEN', 'CCAA_DESTINO', 'PROVINCIA_DESTINO', 'TURISTAS', 'PERNOCTACIONES', 'ESTANCIA_MEDIA']
First 3 rows:
    Unnamed: 0   AÑO  MES CCAA_ORIGEN PROVINCIA_ORIGEN CCAA_DESTINO  \
0           0  2019    7   Andalucía          Almería    Andalucía   

  PROVINCIA_DESTINO TURISTAS PERNOCTACIONES ESTANCIA_MEDIA  
0             Cádiz   8910,0        33536,0            3,8  

 GASTO_TPV_DESTINO_CCAA_MES_DL
Columns: ['Unnamed: 0', 'AÑO', 'MES', 'CCAA_DESTINO', 'TIPO_ORIGEN', 'GASTO', 'TRANSACCION']
First 3 rows:
    Unnamed: 0   AÑO  MES CCAA_DESTINO    TIPO_ORIGEN        GASTO  TRANSACCION
0   

In [112]:
# Para visualizar un archivo en concreto:

datasets["FRONTUR_DL"].head()

,Unnamed: 0,AÑO,MES,PAIS_RESIDENCIA,TIPO_VISITANTE,CCAA_DESTINO,VISITANTES
0,0,2018,1,Alemania,Excursionista,Andalucía,818
1,1,2018,1,Alemania,Excursionista,Baleares,3694
2,2,2018,1,Alemania,Excursionista,Canarias,13587
3,3,2018,1,Alemania,Excursionista,Cataluña,4623
4,4,2018,1,Alemania,Excursionista,Galicia,14


In [113]:
datasets["TURISMO_INTERNO_PROV_CCAA_DL"].head()

,Unnamed: 0,AÑO,MES,CCAA_ORIGEN,PROVINCIA_ORIGEN,CCAA_DESTINO,PROVINCIA_DESTINO,TURISTAS,PERNOCTACIONES,ESTANCIA_MEDIA
0,0,2019,7,Andalucía,Almería,Andalucía,Cádiz,"8910,0","33536,0","3,8"
1,1,2019,7,Andalucía,Almería,Andalucía,Córdoba,"3468,0","11981,0","3,5"
2,2,2019,7,Andalucía,Almería,Andalucía,Granada,"49626,0","141865,0","2,9"
3,3,2019,7,Andalucía,Almería,Andalucía,Huelva,"2801,0","10920,0","3,9"
4,4,2019,7,Andalucía,Almería,Andalucía,Jaén,"9247,0","34155,0","3,7"


In [114]:
datasets["GASTO_TPV_DESTINO_CCAA_MES_DL"].head()

,Unnamed: 0,AÑO,MES,CCAA_DESTINO,TIPO_ORIGEN,GASTO,TRANSACCION
0,0,2019,1,Andalucía,Internacional,"18481196,78",414192
1,1,2019,1,Andalucía,Interregional,"34039254,92",952493
2,2,2019,1,Andalucía,Regional,"33721507,96",1020723
3,3,2019,1,Andalucía,Total Nacional,"67760762,88",1973216
4,4,2019,1,Aragón,Internacional,"503190,68",10783
